# Interactive Deterrence Playground

Use the sliders below to choose parameters and see how total system wealth evolves. Each configuration is averaged across multiple Monte Carlo runs (default: 10; adjustable via the “MC runs” slider) so the curves reflect policy rather than random noise. The chart updates automatically whenever you release a slider.

*Note: Nuclear weapons are not modeled yet—this focuses on conventional deterrence and bargaining.*


In [9]:
import importlib
import contextlib
import io

import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display

import deterrence_simulation as ds

plt.style.use('seaborn-v0_8')



In [10]:
def run_simulation(
    num_countries: int,
    num_rounds: int,
    attack_cost: float,
    failed_attack_cost: float,
    attacker_defense_loss: float,
    defender_defense_loss: float,
    bargain_ev: float,
    trials: int = 10,
    base_seed: int = 42,
):
    """Run multiple simulation trials and return averages for plotting."""
    original_params = {
        'NUM_COUNTRIES': ds.NUM_COUNTRIES,
        'ATTACK_COST_PERCENTAGE': ds.ATTACK_COST_PERCENTAGE,
        'FAILED_ATTACK_COST_PERCENTAGE': ds.FAILED_ATTACK_COST_PERCENTAGE,
        'ATTACKER_DEFENSE_LOSS_PERCENTAGE': ds.ATTACKER_DEFENSE_LOSS_PERCENTAGE,
        'DEFENDER_DEFENSE_LOSS_PERCENTAGE': ds.DEFENDER_DEFENSE_LOSS_PERCENTAGE,
        'BARGAIN_EV_PERCENTAGE': ds.BARGAIN_EV_PERCENTAGE,
    }

    ds.NUM_COUNTRIES = num_countries
    ds.ATTACK_COST_PERCENTAGE = attack_cost
    ds.FAILED_ATTACK_COST_PERCENTAGE = failed_attack_cost
    ds.ATTACKER_DEFENSE_LOSS_PERCENTAGE = attacker_defense_loss
    ds.DEFENDER_DEFENSE_LOSS_PERCENTAGE = defender_defense_loss
    ds.BARGAIN_EV_PERCENTAGE = bargain_ev

    round_count = num_rounds + 1
    totals_matrix = np.zeros((trials, round_count))
    bargains_matrix = np.zeros((trials, num_rounds))
    attacks_matrix = np.zeros((trials, num_rounds))

    try:
        for t in range(trials):
            seed = base_seed + t
            sim = ds.DeterrenceSimulation(num_countries=num_countries, seed=seed)
            totals_matrix[t, 0] = sim.get_total_value()

            for round_idx in range(num_rounds):
                with contextlib.redirect_stdout(io.StringIO()):
                    results = sim.run_round()
                totals_matrix[t, round_idx + 1] = sim.get_total_value()
                bargains_matrix[t, round_idx] = sum(1 for r in results if r['outcome'] == 'bargain')
                attacks_matrix[t, round_idx] = sum(1 for r in results if r['outcome'] != 'bargain')
    finally:
        ds.NUM_COUNTRIES = original_params['NUM_COUNTRIES']
        ds.ATTACK_COST_PERCENTAGE = original_params['ATTACK_COST_PERCENTAGE']
        ds.FAILED_ATTACK_COST_PERCENTAGE = original_params['FAILED_ATTACK_COST_PERCENTAGE']
        ds.ATTACKER_DEFENSE_LOSS_PERCENTAGE = original_params['ATTACKER_DEFENSE_LOSS_PERCENTAGE']
        ds.DEFENDER_DEFENSE_LOSS_PERCENTAGE = original_params['DEFENDER_DEFENSE_LOSS_PERCENTAGE']
        ds.BARGAIN_EV_PERCENTAGE = original_params['BARGAIN_EV_PERCENTAGE']

    rounds = list(range(round_count))
    avg_totals = totals_matrix.mean(axis=0)
    avg_bargains = bargains_matrix.mean(axis=0)
    avg_attacks = attacks_matrix.mean(axis=0)

    pct_change = np.zeros_like(avg_totals)
    pct_change[1:] = np.where(
        avg_totals[:-1] > 0,
        (avg_totals[1:] - avg_totals[:-1]) / avg_totals[:-1] * 100,
        0,
    )

    return rounds, avg_totals, pct_change, avg_bargains, avg_attacks



In [11]:
def plot_totals(num_countries, num_rounds, attack_cost, failed_attack_cost, attacker_defense_loss, defender_defense_loss, bargain_ev, mc_trials):
    rounds, totals, pct_change, bargains, attacks = run_simulation(
        num_countries=int(num_countries),
        num_rounds=int(num_rounds),
        attack_cost=attack_cost,
        failed_attack_cost=failed_attack_cost,
        attacker_defense_loss=attacker_defense_loss,
        defender_defense_loss=defender_defense_loss,
        bargain_ev=bargain_ev,
        trials=int(mc_trials),
    )

    fig, (ax_total, ax_pct) = plt.subplots(2, 1, figsize=(10, 10), sharex=True)

    ax_total.plot(rounds, totals, marker='o', linewidth=2, color='#004488')
    ax_total.set_ylabel('Average Total System Value', fontsize=12, fontweight='bold')
    ax_total.set_title(f'Total Wealth Over Time (averaged over {int(mc_trials)} runs)', fontsize=14, fontweight='bold')
    ax_total.grid(True, alpha=0.3)

    summary_lines = [f"Round {r}: {b:.1f} bargains / {a:.1f} attacks" for r, b, a in zip(rounds[1:], bargains, attacks)]
    annotation = "\n".join(summary_lines) if summary_lines else "No rounds run"
    ax_total.text(1.02, 0.5, annotation, transform=ax_total.transAxes, va='center', fontsize=10,
                  bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

    ax_pct.bar(rounds[1:], pct_change[1:], color='#E67E22', alpha=0.8)
    ax_pct.axhline(0, color='black', linewidth=1)
    ax_pct.set_xlabel('Round', fontsize=12, fontweight='bold')
    ax_pct.set_ylabel('% Change vs Previous Round', fontsize=12, fontweight='bold')
    ax_pct.set_title('Percentage Change in Total Wealth', fontsize=12, fontweight='bold')
    ax_pct.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()



In [12]:
def make_slider(min_val, max_val, step, value, handle_color='#3B82F6'):
    slider = widgets.FloatSlider(min=min_val, max=max_val, step=step, value=value,
                                 readout=False, continuous_update=False,
                                 layout=widgets.Layout(width='220px'))
    slider.style.handle_color = handle_color
    slider.style.description_width = '0px'
    return slider

num_countries_slider = widgets.IntSlider(value=100, min=10, max=200, step=10, readout=False, continuous_update=False,
                                         layout=widgets.Layout(width='220px'))
num_countries_slider.style.handle_color = '#3B82F6'
num_rounds_slider = widgets.IntSlider(value=3, min=1, max=10, step=1, readout=False, continuous_update=False,
                                      layout=widgets.Layout(width='220px'))
num_rounds_slider.style.handle_color = '#3B82F6'
mc_trials_slider = widgets.IntSlider(value=10, min=5, max=100, step=5, readout=False, continuous_update=False,
                                     layout=widgets.Layout(width='220px'))
mc_trials_slider.style.handle_color = '#3B82F6'

attack_cost_slider = make_slider(0.05, 0.30, 0.01, ds.ATTACK_COST_PERCENTAGE)
failed_cost_slider = make_slider(0.20, 0.70, 0.02, ds.FAILED_ATTACK_COST_PERCENTAGE)
attacker_defense_loss_slider = make_slider(0.02, 0.30, 0.01, ds.ATTACKER_DEFENSE_LOSS_PERCENTAGE)
defender_defense_loss_slider = make_slider(0.0, 0.20, 0.01, ds.DEFENDER_DEFENSE_LOSS_PERCENTAGE)
bargain_ev_slider = make_slider(0.02, 0.25, 0.01, ds.BARGAIN_EV_PERCENTAGE)

value_labels = {}


def decorate_slider(label_text, slider, fmt="{:.2f}"):
    value_labels[slider] = widgets.Label(fmt.format(slider.value), layout=widgets.Layout(width='60px'))

    def _update(change):
        value_labels[slider].value = fmt.format(change['new'])

    slider.observe(_update, names='value')
    header = widgets.HBox([
        widgets.Label(label_text, layout=widgets.Layout(width='120px', color='#374151')),
        value_labels[slider]
    ], layout=widgets.Layout(justify_content='space-between', width='220px'))
    return widgets.VBox([header, slider])

controls = {
    'num_countries': num_countries_slider,
    'num_rounds': num_rounds_slider,
    'attack_cost': attack_cost_slider,
    'failed_attack_cost': failed_cost_slider,
    'attacker_defense_loss': attacker_defense_loss_slider,
    'defender_defense_loss': defender_defense_loss_slider,
    'bargain_ev': bargain_ev_slider,
    'mc_trials': mc_trials_slider,
}

param_grid = widgets.GridBox(
    children=[
        decorate_slider('Countries', num_countries_slider, fmt="{}"),
        decorate_slider('Rounds', num_rounds_slider, fmt="{}"),
        decorate_slider('MC runs', mc_trials_slider, fmt="{}"),
        decorate_slider('Attack cost', attack_cost_slider),
        decorate_slider('Failed cost', failed_cost_slider),
        decorate_slider('Attacker loss', attacker_defense_loss_slider),
        decorate_slider('Defender loss', defender_defense_loss_slider),
        decorate_slider('Bargain EV', bargain_ev_slider),
    ],
    layout=widgets.Layout(grid_template_columns='repeat(4, 1fr)', grid_row_gap='12px', grid_column_gap='18px', width='100%')
)

ui = widgets.VBox([
    widgets.HTML('<h4 style="margin-bottom:8px;">Simulation Parameters</h4>'),
    param_grid,
], layout=widgets.Layout(border='1px solid #ddd', padding='12px', border_radius='6px', width='100%'))

output = widgets.interactive_output(plot_totals, controls)

display(ui, output)



Output()